# **Feature engineering dos microdados do ENEM 2022**
#
<ul>
    <li><a href="03%20-%20Feature%20Engineering.ipynb#combina%C3%A7%C3%A3o-de-features">Combinação de Features</a></li>
    <li><a href="03%20-%20Feature%20Engineering.ipynb#considerações-sobre-ponderação-arbitrária">Considerações sobre Ponderação Arbitrária</a></li>
</ul>

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Carrega os dados do arquivo da etapa anterior
file_path = '../data/df_encoded.csv'
df = pd.read_csv(file_path, sep=';', encoding='iso-8859-1')
df

,TP_FAIXA_ETARIA,Q005,Q006,NU_NOTA_LC,NU_NOTA_CH,NU_NOTA_CN,NU_NOTA_MT,NU_NOTA_REDACAO,TP_PRESENCA_LC,TP_PRESENCA_CH,...,TP_ESCOLA_Privada,TP_ESCOLA_Pública,TP_COR_RACA_Amarela,TP_COR_RACA_Branca,TP_COR_RACA_Indígena,TP_COR_RACA_Parda,TP_COR_RACA_Preta,IN_TREINEIRO_Sim,Q024_Sim,Q025_Sim
0,0,5,10,440.6,454.3,394.9,538.2,0.0,Presente,Presente,...,0,1,0,0,0,0,1,0,1,1
1,0,4,11,554.7,584.5,508.0,589.0,740.0,Presente,Presente,...,1,0,0,0,0,0,1,0,1,1
2,1,3,1,NaN,NaN,NaN,NaN,NaN,Faltou,Faltou,...,0,0,0,0,0,1,0,0,1,1
3,8,4,13,618.3,617.3,579.1,638.3,640.0,Presente,Presente,...,0,0,0,1,0,0,0,0,1,1
4,0,2,3,535.3,540.7,493.8,514.8,740.0,Presente,Presente,...,0,1,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,1,5,8,311.9,507.5,574.1,502.8,500.0,Presente,Presente,...,0,1,0,0,0,0,0,0,1,1
49996,0,5,11,585.3,599.8,551.0,657.4,700.0,Presente,Presente,...,0,0,0,1,0,0,0,1,1,1
49997,1,5,8,492.7,443.7,465.5,429.6,500.0,Presente,Presente,...,0,1,0,1,0,0,0,0,0,1
49998,2,6,3,516.4,578.6,490.1,500.8,780.0,Presente,Presente,...,0,1,0,1,0,0,0,0,0,1


## **Combinação de Features**

In [ ]:
df_eng = df.copy()

# Calcula a média das notas objetivas e descarta as colunas individuais
df_eng['media_objetivas'] = df_eng[['NU_NOTA_LC', 'NU_NOTA_CH', 'NU_NOTA_CN', 'NU_NOTA_MT']].mean(axis=1)
df_eng.drop(columns=['NU_NOTA_LC', 'NU_NOTA_CH', 'NU_NOTA_CN', 'NU_NOTA_MT'], inplace=True)

# Combina as informações de faltas (se faltou em qualquer prova recebe 1, caso contrário, 0)
df_eng['faltou'] = df_eng[['TP_PRESENCA_LC', 'TP_PRESENCA_CH', 'TP_PRESENCA_CN', 'TP_PRESENCA_MT']].apply(
    lambda row: any(valor in ['Faltou', 'Eliminado'] for valor in row), axis=1
).astype(int)
df_eng.drop(columns=['TP_PRESENCA_LC', 'TP_PRESENCA_CH', 'TP_PRESENCA_CN', 'TP_PRESENCA_MT'], inplace=True)

# Combina as colunas de raça em duas categorias: branco_amarelo e preto_pardo_indigena
df_eng['branco_amarelo'] = (df_eng['TP_COR_RACA_Branca'] + df_eng['TP_COR_RACA_Amarela']).clip(0, 1)
df_eng['preto_pardo_indigena'] = (df_eng['TP_COR_RACA_Preta'] + df_eng['TP_COR_RACA_Parda'] + df_eng['TP_COR_RACA_Indígena']).clip(0, 1)
df_eng.drop(columns=['TP_COR_RACA_Branca', 'TP_COR_RACA_Amarela', 'TP_COR_RACA_Preta', 'TP_COR_RACA_Parda', 'TP_COR_RACA_Indígena'], inplace=True)

# Junta Q024 (computador) e Q025 (internet) em uma única coluna 'acesso_tecnologia', que recebe 1 se o candidato tem qualquer um dos dois, ou 0 se não tiver nenhum
df_eng['acesso_tecnologia'] = (df_eng['Q024_Sim'] + df_eng['Q025_Sim']).clip(0, 1)
df_eng.drop(columns=['Q024_Sim', 'Q025_Sim'], inplace=True)

# Renomeia as colunas, removendo prefixos e sufixos, convertendo para minúsculas e removendo acentos
renomear = {
    'NU_NOTA_REDACAO': 'redacao',
    'TP_ESCOLA_Privada': 'escola_privada',
    'TP_ESCOLA_Pública': 'escola_publica',
    'TP_SEXO_Feminino': 'sexo_feminino',
    'IN_TREINEIRO_Sim': 'treineiro',
    'Q005': 'tamanho_familia',
    'Q006': 'renda_familiar',
    'TP_FAIXA_ETARIA': 'faixa_etaria'
}
df_eng.rename(columns=renomear, inplace=True)
df_eng.columns = df_eng.columns.str.lower()
df_eng

,faixa_etaria,tamanho_familia,renda_familiar,redacao,sexo_feminino,escola_privada,escola_publica,treineiro,media_objetivas,faltou,branco_amarelo,preto_pardo_indigena,acesso_tecnologia
0,0,5,10,0.0,1,0,1,0,457.000,0,0,1,1
1,0,4,11,740.0,1,1,0,0,559.050,0,0,1,1
2,1,3,1,NaN,1,0,0,0,NaN,1,0,1,1
3,8,4,13,640.0,0,0,0,0,613.250,0,1,0,1
4,0,2,3,740.0,1,0,1,0,521.150,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,1,5,8,500.0,0,0,1,0,474.075,0,0,0,1
49996,0,5,11,700.0,1,0,0,1,598.375,0,1,0,1
49997,1,5,8,500.0,0,0,1,0,457.875,0,1,0,1
49998,2,6,3,780.0,1,0,1,0,521.475,0,1,0,1


## **Considerações sobre Ponderação Arbitrária**

As colunas de renda familiar e faixa etária representam faixas de valores, e não números contínuos. Como cada faixa cobre um intervalo, **existe um grau de ponderação arbitrária ao tratá-las como valores numéricos**. Isso ocorre porque os intervalos não possuem necessariamente a mesma amplitude, e a diferença entre faixas pode não ser linear. No entanto, essa abordagem simplifica a modelagem e evita a explosão dimensional que ocorreria com One Hot Encoding. **Optou-se por esse modelo mais simples para manter a interpretabilidade e a eficiência computacional.**

In [3]:
# Valores da coluna 'renda_familiar':
#  0 → Nenhuma renda
#  1 → Até R$ 1.212,00
#  2 → De R$ 1.212,01 até R$ 1.818,00
#  3 → De R$ 1.818,01 até R$ 2.424,00
#  4 → De R$ 2.424,01 até R$ 3.030,00
#  5 → De R$ 3.030,01 até R$ 3.636,00
#  6 → De R$ 3.636,01 até R$ 4.848,00
#  7 → De R$ 4.848,01 até R$ 6.060,00
#  8 → De R$ 6.060,01 até R$ 7.272,00
#  9 → De R$ 7.272,01 até R$ 8.484,00
# 10 → De R$ 8.484,01 até R$ 9.696,00
# 11 → De R$ 9.696,01 até R$ 10.908,00
# 12 → De R$ 10.908,01 até R$ 12.120,00
# 13 → De R$ 12.120,01 até R$ 14.544,00
# 14 → De R$ 14.544,01 até R$ 18.180,00
# 15 → De R$ 18.180,01 até R$ 24.240,00
# 16 → Mais de R$ 24.240,00

# Valores da coluna 'faixa_etaria':
#  0 → Menor de 17 anos
#  1 → 17 anos
#  2 → 18 anos
#  3 → 19 anos
#  4 → 20 anos
#  5 → 21 anos
#  6 → 22 anos
#  7 → 23 anos
#  8 → 24 anos
#  9 → 25 anos
# 10 → Entre 26 e 30 anos
# 11 → Entre 31 e 35 anos
# 12 → Entre 36 e 40 anos
# 13 → Entre 41 e 45 anos
# 14 → Entre 46 e 50 anos
# 15 → Entre 51 e 55 anos
# 16 → Entre 56 e 60 anos
# 17 → Entre 61 e 65 anos
# 18 → Entre 66 e 70 anos
# 19 → Maior de 70 anos

In [4]:
# Salva o dataframe para a etapa seguinte (modelagem)
df_eng.to_csv('../data/df_eng.csv', sep=';', encoding='iso-8859-1', index=False)